In [63]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [64]:
csv_file = "athlete_events.csv"
olypmic_df = pd.read_csv(csv_file)
olypmic_df = olypmic_df.drop(['Weight'], axis=1)

In [65]:
olypmic_df.Height=olypmic_df.Height.fillna(value=0)

In [66]:
countires = olypmic_df.Team.unique()
countryNoc =  olypmic_df.NOC.unique()
olypmic_df.Medal=olypmic_df.Medal.fillna(value="no_medal")

In [67]:
country_gold_list = list()
country_silver_list = list()
country_bronze_list = list()

for val in countryNoc:
    newdf = olypmic_df.loc[olypmic_df.NOC==val]
    country_gold_list.append(len(newdf.loc[newdf.Medal=="Gold"]))
for val in countryNoc:
    newdf = olypmic_df.loc[olypmic_df.NOC==val]
    country_silver_list.append(len(newdf.loc[newdf.Medal=="Silver"]))
for val in countryNoc:
    newdf = olypmic_df.loc[olypmic_df.NOC==val]
    country_bronze_list.append(len(newdf.loc[newdf.Medal=="Bronze"]))

print(len(country_gold_list),len(country_silver_list))

230 230


In [68]:
country_medal_df = pd.DataFrame(countryNoc)
country_medal_df["Gold_Medals"]=country_gold_list
country_medal_df["Silver_Medals"]=country_silver_list
country_medal_df["Bronze_Medals"]=country_bronze_list

country_medal_df=country_medal_df.rename(columns={0:"NOC"})
country_medal_df["Total_Medals"]=country_medal_df.sum(axis=1)
country_medal_df

,NOC,Gold_Medals,Silver_Medals,Bronze_Medals,Total_Medals
0,CHN,350,347,292,989
1,DEN,179,241,177,597
2,NED,287,340,413,1040
3,USA,2638,1641,1358,5637
4,FIN,198,270,432,900
...,...,...,...,...,...
225,TUV,0,0,0,0
226,NFL,0,0,0,0
227,KOS,1,0,0,1
228,SSD,0,0,0,0


In [69]:
heights_df = olypmic_df[olypmic_df.Height>0]
heights_m_df =  heights_df[heights_df.Sex=="M"]
heights_f_df =  heights_df[heights_df.Sex=="F"]

In [70]:
m_avg_h=heights_m_df.groupby("Year")["Height"].mean()
f_avg_h=heights_f_df.groupby("Year")["Height"].mean()
mheight_df =  pd.DataFrame(m_avg_h)
fheight_df =  pd.DataFrame(f_avg_h)

In [71]:
mheight_df['Female_Height']=fheight_df.Height
combined_height=mheight_df.rename(columns={"Height":"Male_Height"})

In [72]:
#olypmic_df.to_csv("olympic_data.csv")
#country_medal_df.to_csv("country_medal.csv")
#combined_height.to_csv("combined_height_avg.csv")
combined_height=combined_height.rename(columns={"Year":"year","Male_Height":"male_height","Female_Height":"female_height"})

In [79]:
x=len(combined_height.male_height)
val=[]
t=0
while t < x:
    val.append(t+1)
    t+=1
val
combined_height["id"]=val
combined_height["year"]=combined_height.index
combined_height=combined_height.set_index('id')
country_medal_df.head()

,NOC,Gold_Medals,Silver_Medals,Bronze_Medals,Total_Medals
0,CHN,350,347,292,989
1,DEN,179,241,177,597
2,NED,287,340,413,1040
3,USA,2638,1641,1358,5637
4,FIN,198,270,432,900


In [80]:
val=[]
x = len(country_medal_df.NOC)
val=[]
t=0
while t<x:
    val.append(t+1)
    t+=1
country_medal_df['id']=val
country_medal_df=country_medal_df.set_index('id')
country_medal_df.head()

,NOC,Gold_Medals,Silver_Medals,Bronze_Medals,Total_Medals
id,,,,,
1,CHN,350,347,292,989
2,DEN,179,241,177,597
3,NED,287,340,413,1040
4,USA,2638,1641,1358,5637
5,FIN,198,270,432,900


In [74]:
rds_connection_string = "postgres:postgres@localhost:5432/olympic_stats"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [75]:
engine.table_names()

['country_medal_stats', 'combined_height']

In [76]:
combined_height.to_sql(name='combined_height_average', con=engine, if_exists='append', index=True)

In [81]:
country_medal_df.to_sql(name='country_medal_stats', con=engine, if_exists='append', index=True)